This code was adapted from:


*   NLP Labs


In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

drive.mount("/content/drive")

data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 ArtsRatings_5000_test.txt	    JewelleryReviewsLSA.csv
 ArtsRatings_5000_train.txt	    JewelleryReviewsQueryRelevantID.csv
 ArtsReviews_5000_test.txt	    metamorphosis_clean.txt
 ArtsReviews_5000_train.txt	   'My CW1.ipynb'
'F135864 - Task 1.ipynb'	   'My CW 2.ipynb'
'F135864 - Task 2.ipynb'	   'MY CW 3.ipynb'
'F135864 - Task 3.ipynb'	    Reduced_ArtsRatings_5000.txt
'F135864 - Task 4.ipynb'	    Reduced_ArtsReviews_5000.txt
 glove.6B.100d.txt		    review_polarity
 JewelleryReviewsDeepLearning.csv   Untitled0.ipynb


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/JewelleryReviewsDeepLearning.csv')
reviews = df["Reviews"].to_numpy()
ratings = df["Ratings"].to_numpy()


In [ ]:
#Merging reviews with all ratings 1,2,3,4,5 while drecreasing limit to account for model mamixmum number of input
batch_1 = df[:1000]
batch_2 = df[2000:3000]
batch_3 = df[4000:5000]
batch_4 = df[6000:7000]
batch_5 = df[8000:9000]

frames = [batch_1, batch_2, batch_3,batch_4, batch_5]
batch = pd.concat(frames)


In [ ]:
batch

,ID,Reviews,Ratings
0,8,I have generally had very good experience orde...,1
1,21,"when i opened the box, i was expecting two ele...",1
2,59,I ordered a black titanium barbell. Once i rec...,1
3,60,I had ordered this for my girlfriends birthday...,1
4,83,I really am confused in why this item got so m...,1
...,...,...,...
8995,24689,"Look beautiful, but had to return it too small...",4
8996,24695,"In the Parable of the Mustard Seed, Jesus comp...",4
8997,24723,I like the charm and it got here quickly - shi...,4
8998,24725,So cute. I ordered this a couple years ago for...,4


In [ ]:
batch['Ratings'].value_counts()

4    2000
1    1000
2    1000
3    1000
Name: Ratings, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(type(batch['Reviews']))
print(batch['Reviews'].shape)

<class 'pandas.core.series.Series'>
(5000,)


In [ ]:
tokenized = batch['Reviews'].apply((lambda x: tokenizer.encode(x, padding=True, truncation=True,max_length=50, add_special_tokens=True)))
print(tokenized)

0       [101, 1045, 2031, 3227, 2018, 2200, 2204, 3325...
1       [101, 2043, 1045, 2441, 1996, 3482, 1010, 1045...
2       [101, 1045, 3641, 1037, 2304, 23431, 3347, 173...
3       [101, 1045, 2018, 3641, 2023, 2005, 2026, 2740...
4       [101, 1045, 2428, 2572, 5457, 1999, 2339, 2023...
                              ...                        
8995    [101, 2298, 3376, 1010, 2021, 2018, 2000, 2709...
8996    [101, 1999, 1996, 11498, 3468, 1997, 1996, 231...
8997    [101, 1045, 2066, 1996, 11084, 1998, 2009, 228...
8998    [101, 2061, 10140, 1012, 1045, 3641, 2023, 103...
8999    [101, 3835, 26261, 4892, 24044, 1010, 2022, 80...
Name: Reviews, Length: 5000, dtype: object


Padding

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print(padded)

[[  101  1045  2031 ...  6910  2006   102]
 [  101  2043  1045 ...     0     0     0]
 [  101  1045  3641 ... 20051  2378   102]
 ...
 [  101  1045  2066 ...  2228  2009   102]
 [  101  2061 10140 ...  1010  1045   102]
 [  101  3835 26261 ...  4677  2007   102]]


In [ ]:
np.array(padded).shape

(5000, 50)

Masking

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 50)

DistilBert

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch['Ratings']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
print(train_features.shape)
print(test_features.shape)

(3750, 768)
(1250, 768)


In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.5504

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.400 (+/- 0.00)


The model scored an adequate score in the evaluation, 0.15 higher than Dummy classifier score. If fine-tuning is added to the model, it can reach an accuracy of 0.7 or higher